In [21]:
print('경사법 예시')
import numpy as np

def function_2(x):
    return np.sum(x**2)

def numerical_gradient(f,x):
    h = 1e-4 #0.0001
    grad = np.zeros_like(x)  #x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        #f(x + h)계산
        x[idx] = tmp_val +h
        fxh1 = f(x)
        
        #f(x - h)계산
        x[idx] = tmp_val -h
        fxh2 = f(x)  
        
        grad[idx] = (fxh1 - fxh2)/(2*h)
        x[idx] = tmp_val  #값 복원
        
    return grad


print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([0.0, 2.0])))
print(numerical_gradient(function_2, np.array([3.0, 0.0])))
    

경사법 예시
[ 0.  4.]
[ 0.  4.]
[ 6.  0.]


In [19]:
print('경사 하강법 예시')

def gradient_descent(f, init_x, lr =0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr*grad
        
    return x

print('-3.0, 4.0에서 경사법으로 최소값 탐색')
init_x = np.array([-3.0, 4.0])
print(gradient_descent(function_2, init_x=init_x, lr =0.1, step_num=100))

print('학습률이 너무 큰 예: lr =10.0인 경우')
print(gradient_descent(function_2, init_x=init_x, lr =10.0, step_num=100))

print('학습률이 너무 작은 예: lr =1e-10인 경우')
print(gradient_descent(function_2, init_x=init_x, lr =1e-10, step_num=100))

경사 하강법 예시
-3.0, 4.0에서 경사법으로 최소값 탐색
[ -6.11110793e-10   8.14814391e-10]
학습률이 너무 큰 예: lr =10.0인 경우
[  2.34235971e+12  -3.96091057e+12]
학습률이 너무 작은 예: lr =1e-10인 경우
[  2.34235971e+12  -3.96091057e+12]


In [1]:
print('신경망 기울기 예시')
import numpy as np

#softmaxt function, cross_entropy_error function, numerical_gradient function
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c) #오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def numerical_gradient(f, x):  #기존의 책의 내용과 차이가 있음! 주의!
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)

        x[idx] = tmp_val - h
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)

        x[idx] = tmp_val # 값 복원
        it.iternext()

    return grad


#기울기를 구하기 위한 환경 class설정
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) #정규분포로 초기화
        
    def predict(self,x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z= self.predict(x)
        y= softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss
    
    
net = simpleNet()
print(net.W)  #가중치 매개변수
print('\n')

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
print('\n')
np.argmax(p)  #최대값의 인덱스

t = np.array([0, 0,1])  #정답 레이블
print(net.loss(x,t))
print('\n')


# 기울기 구하기
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

신경망 기울기 예시
[[ 1.77111385  0.96331216 -0.74215061]
 [ 0.18331289  0.59383651 -1.56567128]]


[ 1.22764991  1.11244016 -1.85439452]


3.74320360953


[[ 0.30974892  0.27604179 -0.58579071]
 [ 0.46462338  0.41406269 -0.87868607]]
